In [47]:
%reload_ext autoreload
%autoreload 2

In [48]:
import src.config as config

In [49]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28) #28

current_date=Timestamp('2023-10-05 07:00:00')


In [50]:
# current_date - timedelta(days=71)

In [51]:
# fetch_data_from - timedelta(days=7*52)

In [52]:
# prueba = fetch_data_from - timedelta(days=7*52)
# prueba.month

# from src.data import load_raw_data
# prueba = load_raw_data(year=2022, months=8)
# prueba

In [53]:
# fetch_data_from - timedelta(days=7*52)

In [54]:
# apoyo = fetch_data_from - timedelta(days=7*52)
# pruebe = prueba[prueba.pickup_datetime >=  apoyo]
# prueba

In [55]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # # download 2 files from website
    # rides = load_raw_data(year=from_date_.year) #, months=from_date_.month)
    # rides = rides[(rides.pickup_datetime >= from_date_) & (rides.pickup_datetime < to_date_)]
   
    # rides_2 = load_raw_data(year=to_date_.year) #, months=to_date_.month)
    # rides_2 = rides_2[(rides_2.pickup_datetime < to_date_) & (rides_2.pickup_datetime < to_date_)]

    # rides = pd.concat([rides, rides_2]) 

    # Intenta cargar los datos del primer año
    rides = load_raw_data(year=from_date_.year)
    rides = rides[(rides.pickup_datetime >= from_date_) & (rides.pickup_datetime < to_date_)]

    # Verifica si los años son diferentes
    if from_date_.year != to_date_.year:
    # Carga los datos del segundo año
        rides_2 = load_raw_data(year=to_date_.year)
        rides_2 = rides_2[(rides_2.pickup_datetime >= from_date_) & (rides_2.pickup_datetime < to_date_)]
        rides = pd.concat([rides, rides_2]) 


    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [56]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2023-09-07 07:00:00'), to_date_=Timestamp('2022-10-06 07:00:00')
File 2022 was already in local storage


In [57]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
100%|██████████| 319/319 [00:00<00:00, 327.54it/s]


In [58]:
#Falta agregar lo de filtrar la fecha que corresponde, o quitar los meses que no queremos

In [59]:
rides.head()

,pickup_datetime,pickup_location_id,pickup_hour
1909438,2023-09-07 07:35:40,2,2023-09-07 07:00:00
2049010,2023-09-07 07:39:00,2,2023-09-07 07:00:00
1867302,2023-09-07 07:49:14,2,2023-09-07 07:00:00
1820092,2023-09-07 07:50:59,2,2023-09-07 07:00:00
2064924,2023-09-07 07:52:35,2,2023-09-07 07:00:00


In [60]:
ts_data.tail()

,pickup_hour,rides,pickup_location_id
214363,2023-10-05 02:00:00,0,498
214364,2023-10-05 03:00:00,0,498
214365,2023-10-05 04:00:00,0,498
214366,2023-10-05 05:00:00,0,498
214367,2023-10-05 06:00:00,0,498


In [61]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/100501
Connected. Call `.close()` to terminate connection gracefully.


In [62]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

Uploading Dataframe: 0.00% |          | Rows 0/214368 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/100501/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x1c759b98040>, None)